In [30]:
# Optional installs — run if missing
# !pip install torch torchvision pillow efficientnet-pytorch sentence-transformers requests tensorflow
# If you use Ollama locally, ensure it's installed separately (not pip).


In [31]:
import json
import requests
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision.models import efficientnet_b0
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


✅ Using device: cuda


In [32]:
# ------------ Structured Knowledge Base (DICTIONARY) -------------

KNOWLEDGE = {
    "Diseases": {
        "Pepper Bell Bacterial Spot": {
            "Symptoms": "Small dark water-soaked spots on leaves and fruits, yellow halos, leaf drop.",
            "Cause": "Xanthomonas bacteria.",
            "Organic Control": "Neem oil (5 ml/L), copper spray (2 g/L).",
            "Chemical Control": "Copper oxychloride (3 g/L), streptomycin sprays.",
            "Prevention": "Disease-free seeds, crop rotation, avoid overhead watering.",
            "Fertilizer Recommendation": "Balanced NPK 10-10-10.",
            "Climate Conditions": "High humidity, warm temperature (25–30°C).",
            "Confusion With": "Magnesium deficiency, sunscald."
        },

        "Pepper Bell Healthy": {
            "Condition": "Normal deep-green leaves, strong stems.",
            "Fertilizer Recommendation": "NPK 12-12-17 every 25 days.",
            "Prevention": "Proper irrigation, balanced fertilizer."
        },

        "Potato Early Blight": {
            "Symptoms": "Brown concentric rings on lower leaves, yellowing.",
            "Cause": "Alternaria solani.",
            "Organic Control": "Neem oil (3 ml/L).",
            "Chemical Control": "Mancozeb (2.5 g/L).",
            "Prevention": "Crop rotation, avoid overhead irrigation.",
            "Fertilizer Recommendation": "NPK 5-10-20."
        },

        "Potato Late Blight": {
            "Symptoms": "Dark greasy lesions, white mold under leaves, tuber rot.",
            "Cause": "Phytophthora infestans.",
            "Organic Control": "Neem oil (4 ml/L), copper oxychloride (3 g/L).",
            "Chemical Control": "Mancozeb, Metalaxyl.",
            "Prevention": "Remove infected plants, avoid water stagnation.",
            "Climate Conditions": "Cool + humid weather."
        },

        "Potato Healthy": {
            "Condition": "Dark green leaves, strong stems.",
            "Prevention": "Adequate nitrogen, well-drained soil.",
            "Fertilizer Recommendation": "NPK 13-40-13 at planting → 20-20-20."
        },

        "Tomato Bacterial Spot": {
            "Symptoms": "Small dark leaf spots, scabby fruits.",
            "Cause": "Xanthomonas bacteria.",
            "Organic Control": "Neem oil, copper sprays.",
            "Chemical Control": "Copper fungicides.",
            "Prevention": "Avoid overhead watering.",
            "Fertilizer Recommendation": "NPK 5-10-10."
        },

        "Tomato Early Blight": {
            "Symptoms": "Concentric brown rings, leaf yellowing.",
            "Cause": "Alternaria solani.",
            "Organic Control": "Compost tea, neem oil.",
            "Chemical Control": "Mancozeb, chlorothalonil.",
            "Prevention": "Remove infected leaves.",
            "Fertilizer Recommendation": "NPK 10-20-20."
        },

        "Tomato Late Blight": {
            "Symptoms": "Greasy lesions, white mold under leaves.",
            "Cause": "Phytophthora infestans.",
            "Organic Control": "Neem, copper oxychloride.",
            "Chemical Control": "Mancozeb, Metalaxyl.",
            "Prevention": "Remove infected plants.",
            "Fertilizer Recommendation": "Increase potassium."
        },

        "Tomato Leaf Mold": {
            "Symptoms": "Yellow patches on top of leaf, olive mold underside.",
            "Cause": "Passalora fulva.",
            "Organic Control": "Pruning, ventilation, neem spray.",
            "Chemical Control": "Copper or chlorothalonil fungicides.",
            "Prevention": "Reduce humidity."
        },

        "Tomato Septoria Leaf Spot": {
            "Symptoms": "Numerous small brown spots with pale centers.",
            "Cause": "Septoria lycopersici.",
            "Organic Control": "Neem spray.",
            "Chemical Control": "Copper fungicide.",
            "Prevention": "Avoid overhead watering."
        },

        "Tomato Spider Mites": {
            "Symptoms": "Leaf bronzing, tiny webs, yellow speckles.",
            "Cause": "Mites.",
            "Organic Control": "Neem oil (2 ml/L), soap spray.",
            "Prevention": "Increase humidity."
        },

        "Tomato Target Spot": {
            "Symptoms": "Circular spots with rings.",
            "Cause": "Corynespora cassiicola.",
            "Organic Control": "Neem, sanitation.",
            "Chemical Control": "Fungicides.",
            "Prevention": "Good airflow."
        },

        "Tomato Mosaic Virus": {
            "Symptoms": "Mottled leaves, twisted shape.",
            "Cause": "Tomato mosaic virus.",
            "Organic Control": "Remove infected plants.",
            "Prevention": "Sanitize tools."
        },

        "Tomato Yellow Leaf Curl Virus": {
            "Symptoms": "Yellow curled leaves, stunting.",
            "Cause": "Whitefly-transmitted virus.",
            "Organic Control": "Sticky traps, neem oil.",
            "Prevention": "Remove infected plants."
        },

        "Tomato Healthy": {
            "Condition": "Healthy green leaves, no visible spots, good vigor.",
            "Prevention": "Proper irrigation, balanced fertilizer, good spacing.",
            "Fertilizer Recommendation": "NPK 12-12-36 during fruiting stage.",
            "Climate Conditions": "Warm, moderate humidity."
        }
    },

    "Deficiencies": {
        "Nitrogen Deficiency": {
            "Symptoms": "Pale yellow older leaves.",
            "Fix": "Compost, cow manure, urea (1 tsp/L).",
            "Prevention": "Regular compost application."
        },
        "Phosphorus Deficiency": {
            "Symptoms": "Purplish leaves, slow growth.",
            "Fix": "DAP, bone meal.",
            "Prevention": "Maintain soil pH 6–7."
        },
        "Potassium Deficiency": {
            "Symptoms": "Brown leaf edges (scorching).",
            "Fix": "MOP fertilizer.",
            "Prevention": "Balanced NPK schedule."
        },
        "Calcium Deficiency": {
            "Symptoms": "Blossom end rot.",
            "Fix": "Calcium nitrate (1 tsp/L).",
            "Prevention": "Avoid irregular watering."
        },
        "Magnesium Deficiency": {
            "Symptoms": "Yellowing between veins on older leaves.",
            "Fix": "Epsom salt spray (1 tsp/L).",
            "Prevention": "Maintain soil pH 6–7."
        },
        "Iron Deficiency": {
            "Symptoms": "Yellowing between veins on young leaves.",
            "Fix": "Ferrous sulfate spray.",
            "Prevention": "Avoid alkaline soil."
        }
    },

    "Pests": {
        "Aphids": {
            "Symptoms": "Sticky leaves (honeydew), curling.",
            "Organic Control": "Neem oil (2 ml/L), soap spray.",
            "Chemical Control": "Imidacloprid."
        },
        "Whiteflies": {
            "Symptoms": "Tiny white insects flying when disturbed.",
            "Organic Control": "Yellow sticky traps.",
            "Chemical Control": "Thiamethoxam."
        },
        "Thrips": {
            "Symptoms": "Silver streaks, leaf distortion.",
            "Control": "Spinosad, neem spray."
        },
        "Mealybugs": {
            "Symptoms": "Cotton-like clusters.",
            "Organic Control": "Spirit + soap spray."
        },
        "Fruit Worms": {
            "Symptoms": "Holes in fruits.",
            "Control": "Bt spray (Bacillus thuringiensis)."
        }
    },

    "Stages": {
        "Seedling": {"Fertilizer": "NPK 19-19-19 (half dose).", "Care": "Avoid overwatering."},
        "Vegetative": {"Fertilizer": "NPK 20-20-20.", "Care": "Watch for fungal infections."},
        "Flowering": {"Fertilizer": "NPK 15-30-15.", "Care": "Maintain calcium."},
        "Fruiting": {"Fertilizer": "NPK 12-12-17 or 14-14-21.", "Care": "Monitor pests."}
    },

    "Climate": {
        "High Humidity": {"Risk": "Leaf mold, late blight.", "Advice": "Improve ventilation."},
        "Hot Dry": {"Risk": "Spider mites.", "Advice": "Increase humidity."},
        "Cool Wet": {"Risk": "Late blight.", "Advice": "Use protective fungicide spray."}
    },

    "Dosages": {
        "Neem Oil": "2–5 ml per liter.",
        "Copper Oxychloride": "2–3 g per liter.",
        "Mancozeb": "2–2.5 g per liter.",
        "Metalaxyl": "Use as per label.",
        "Imidacloprid": "0.3 ml per liter.",
        "Epsom Salt": "1 tsp per liter.",
        "Calcium Nitrate": "1 tsp per liter."
    }
}

print("✅ Knowledge dictionary created. Diseases:", len(KNOWLEDGE["Diseases"]))


✅ Knowledge dictionary created. Diseases: 15


In [33]:
INSECT_KNOWLEDGE = {
    "Africanized Honey Bees (Killer Bees)": {
        "Symptoms": "Highly aggressive swarming near hives or disturbed areas; bees may chase humans or animals long distances.",
        "Cause": "Hybrid of African and European honey bee subspecies.",
        "Organic Control": "Avoid provoking colonies; contact trained bee removal experts.",
        "Chemical Control": "Not recommended due to environmental impact.",
        "Prevention": "Seal cracks in buildings; remove abandoned hives; maintain distance from colonies.",
        "Climate Conditions": "More active in warm climates.",
        "Confusion With": "European honey bees."
    },

    "Aphids": {
        "Symptoms": "Sticky honeydew on leaves, curled leaves, yellowing, presence of ants.",
        "Cause": "Sap-sucking insects feeding on new growth.",
        "Organic Control": "Neem oil (2 ml/L), soap spray, release ladybugs.",
        "Chemical Control": "Imidacloprid (use only if severe).",
        "Prevention": "Avoid over-fertilizing with nitrogen; use reflective mulches.",
        "Climate Conditions": "Warm, dry weather accelerates reproduction.",
        "Confusion With": "Whiteflies, thrips."
    },

    "Armyworms": {
        "Symptoms": "Skeletonized leaves, chewing marks, defoliation in patches.",
        "Cause": "Larvae of moth species feeding on foliage.",
        "Organic Control": "Bt spray (Bacillus thuringiensis), neem spray.",
        "Chemical Control": "Lambda-cyhalothrin or emamectin benzoate.",
        "Prevention": "Deep plowing to expose pupae; pheromone traps.",
        "Climate Conditions": "Warm, humid weather encourages outbreaks.",
        "Confusion With": "Cutworms, fall armyworms."
    },

    "Brown Marmorated Stink Bugs": {
        "Symptoms": "Sunken spots on fruits, distortion, foul odor when crushed.",
        "Cause": "Piercing–sucking insects feeding on plant tissues.",
        "Organic Control": "Hand removal; trap crops; neem + soap spray.",
        "Chemical Control": "Pyrethroids (use sparingly).",
        "Prevention": "Seal home entry points; remove weeds.",
        "Climate Conditions": "Active in warm temperatures.",
        "Confusion With": "Green stink bugs."
    },

    "Cabbage Loopers": {
        "Symptoms": "Large irregular holes in cabbage, cauliflower, or leafy greens.",
        "Cause": "Caterpillars feeding on leaves.",
        "Organic Control": "Bt spray, neem oil, hand-picking.",
        "Chemical Control": "Spinosad.",
        "Prevention": "Floating row covers; remove decaying vegetation.",
        "Climate Conditions": "Peak damage in warm seasons.",
        "Confusion With": "Diamondback moth larvae."
    },

    "Citrus Canker": {
        "Symptoms": "Raised brown lesions with yellow halos on citrus leaves and fruit.",
        "Cause": "Bacterial infection spread by insects (not an insect itself).",
        "Organic Control": "Copper sprays; prune infected branches.",
        "Chemical Control": "Copper-based bactericides.",
        "Prevention": "Sanitation; resistant varieties.",
        "Climate Conditions": "Thrives in humid tropical climates.",
        "Confusion With": "Leaf miner damage."
    },

    "Colorado Potato Beetles": {
        "Symptoms": "Defoliation of potato leaves; striped adult beetles; orange larvae.",
        "Cause": "Leaf-feeding beetles targeting solanaceous crops.",
        "Organic Control": "Neem oil, hand-picking, Bt var. tenebrionis.",
        "Chemical Control": "Spinosad, imidacloprid (rotate chemicals).",
        "Prevention": "Crop rotation; mulch; trap crops.",
        "Climate Conditions": "Active in warm dry weather.",
        "Confusion With": "Ladybug larvae."
    },

    "Corn Borers": {
        "Symptoms": "Holes in stems, frass deposits, poor grain fill.",
        "Cause": "Larvae boring into corn stalks.",
        "Organic Control": "Bt maize, Bt spray, parasitic wasps.",
        "Chemical Control": "Pyrethroids.",
        "Prevention": "Destroy crop residue; pheromone traps.",
        "Climate Conditions": "Warm climate favors larval survival.",
        "Confusion With": "Armyworms."
    },

    "Corn Earworms": {
        "Symptoms": "Damage to corn ears; kernels eaten; larvae present at tips.",
        "Cause": "Moth larvae feeding on developing corn.",
        "Organic Control": "Bt spray, neem oil.",
        "Chemical Control": "Spinosad, synthetic pyrethroids.",
        "Prevention": "Use tight husk varieties; pheromone trapping.",
        "Climate Conditions": "Hot summers promote infestations.",
        "Confusion With": "Fall armyworms."
    },

    "Fall Armyworms": {
        "Symptoms": "Windowpane feeding, circular patches of dead grass, leaf tattering.",
        "Cause": "Larval feeding by Spodoptera frugiperda.",
        "Organic Control": "Neem oil, Bt, NPV virus spray.",
        "Chemical Control": "Emamectin benzoate, chlorantraniliprole.",
        "Prevention": "Light traps; early planting.",
        "Climate Conditions": "Spread rapidly in warm humid weather.",
        "Confusion With": "Armyworms, cutworms."
    },

    "Fruit Flies": {
        "Symptoms": "Fruit rotting, maggots inside fruit, premature fruit drop.",
        "Cause": "Female flies laying eggs inside fruits.",
        "Organic Control": "Banana-bait traps, neem spray.",
        "Chemical Control": "Malathion bait spray.",
        "Prevention": "Remove fallen fruits; bagging fruits.",
        "Climate Conditions": "Active in warm tropical climate.",
        "Confusion With": "Vinegar flies."
    },

    "Spider Mites": {
        "Symptoms": "Fine webs, yellow speckles, bronzing leaves.",
        "Cause": "Tiny arachnids sucking plant sap.",
        "Organic Control": "Humidify plants, neem oil (2 ml/L), soap spray.",
        "Chemical Control": "Abamectin (if severe).",
        "Prevention": "Avoid dusty/dry conditions.",
        "Climate Conditions": "Explode in hot, dry weather.",
        "Confusion With": "Thrips."
    },

    "Thrips": {
        "Symptoms": "Silvery streaks, leaf distortion, black fecal dots.",
        "Cause": "Slender insects rasping leaves and petals.",
        "Organic Control": "Blue sticky traps, neem spray, predatory mites.",
        "Chemical Control": "Spinosad.",
        "Prevention": "Avoid water stress; weed removal.",
        "Climate Conditions": "Hot, dry weather encourages reproduction.",
        "Confusion With": "Spider mites."
    },

    "Tomato Hornworms": {
        "Symptoms": "Large chewed leaves, missing fruit sections, green caterpillars with horns.",
        "Cause": "Larvae of hawk moth species.",
        "Organic Control": "Hand-picking, neem oil, Bt.",
        "Chemical Control": "Spinosad.",
        "Prevention": "Encourage parasitic wasps.",
        "Climate Conditions": "Thrives in warm climates.",
        "Confusion With": "Armyworms."
    },

    "Western Corn Rootworms": {
        "Symptoms": "Root pruning, lodging of corn plants, yield loss.",
        "Cause": "Larvae feeding on corn roots.",
        "Organic Control": "Crop rotation; soil-applied Bt.",
        "Chemical Control": "Chlorpyrifos (restricted), neonicotinoids.",
        "Prevention": "Rotate crops every year.",
        "Climate Conditions": "Warm continental climate.",
        "Confusion With": "White grubs."
    }
}

print("✅ Insect knowledge loaded. Insects:", len(INSECT_KNOWLEDGE))


✅ Insect knowledge loaded. Insects: 15


In [34]:
CLASS_TO_TITLE = {
    "Pepper___Bacterial_spot": "Pepper Bell Bacterial Spot",
    "Pepper___Healthy": "Pepper Bell Healthy",
    "Potato___Early_blight": "Potato Early Blight",
    "Potato___Late_blight": "Potato Late Blight",
    "Potato___Healthy": "Potato Healthy",
    "Tomato___Bacterial_spot": "Tomato Bacterial Spot",
    "Tomato___Early_blight": "Tomato Early Blight",
    "Tomato___Late_blight": "Tomato Late Blight",
    "Tomato___Leaf_Mold": "Tomato Leaf Mold",
    "Tomato___Septoria_leaf_spot": "Tomato Septoria Leaf Spot",
    "Tomato___Spider_mites": "Tomato Spider Mites",
    "Tomato___Target_Spot": "Tomato Target Spot",
    "Tomato___Mosaic_virus": "Tomato Mosaic Virus",
    "Tomato___Yellow_Leaf_Curl_Virus": "Tomato Yellow Leaf Curl Virus",
    "Tomato___Healthy": "Tomato Healthy"
}


In [35]:
class OfflineAgriculturalRAG:
    def __init__(self, knowledge):
        self.kb = knowledge

    def get_disease_block(self, cnn_class, class_map):
        title = class_map.get(cnn_class)
        if not title:
            return {}
        block = dict(self.kb["Diseases"].get(title, {}))
        block["name"] = title
        return block


class InsectRAG:
    def query(self, insect_name):
        return INSECT_KNOWLEDGE.get(insect_name, {})


In [36]:
import torch
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from PIL import Image

PLANT_CKPT = "checkpoints/efficientnet_b0_best.pth"

# Load checkpoint
plant_ckpt = torch.load(PLANT_CKPT, map_location=device)

plant_classes = plant_ckpt["classes"]
num_classes = len(plant_classes)

# ✅ CORRECT MODEL (efficientnet-pytorch)
plant_model = EfficientNet.from_name(
    "efficientnet-b0",
    num_classes=num_classes
)

plant_model.load_state_dict(plant_ckpt["model"])
plant_model.to(device)
plant_model.eval()

plant_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ Plant CNN loaded correctly (efficientnet-pytorch)")


✅ Plant CNN loaded correctly (efficientnet-pytorch)


In [37]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

INSECT_CLASSES = [
    'Africanized Honey Bees (Killer Bees)',
    'Aphids',
    'Armyworms',
    'Brown Marmorated Stink Bugs',
    'Cabbage Loopers',
    'Citrus Canker',
    'Colorado Potato Beetles',
    'Corn Borers',
    'Corn Earworms',
    'Fall Armyworms',
    'Fruit Flies',
    'Spider Mites',
    'Thrips',
    'Tomato Hornworms',
    'Western Corn Rootworms'
]

INSECT_CKPT = "efficientnet_insects.pth"

# 1️⃣ Build torchvision EfficientNet-B0
insect_model = efficientnet_b0(weights=None)

# 2️⃣ Replace classifier head (VERY IMPORTANT)
insect_model.classifier[1] = nn.Linear(
    insect_model.classifier[1].in_features,
    len(INSECT_CLASSES)
)

# 3️⃣ Load your checkpoint (this will now MATCH)
state_dict = torch.load(INSECT_CKPT, map_location=device)
insect_model.load_state_dict(state_dict)

# 4️⃣ Finalize
insect_model.to(device).eval()

print("✅ Insect CNN loaded correctly (torchvision EfficientNet)")


✅ Insect CNN loaded correctly (torchvision EfficientNet)


In [38]:
def get_weather(city="Hyderabad"):
    try:
        url = f"https://wttr.in/{city}?format=j1"
        data = requests.get(url, timeout=5).json()
        current = data["current_condition"][0]
        return f"{current['weatherDesc'][0]['value']}, {current['temp_C']}°C"
    except:
        return "Weather unavailable"


In [39]:
class AgroAI:
    def __init__(self):
        self.plant_rag = OfflineAgriculturalRAG(KNOWLEDGE)
        self.insect_rag = InsectRAG()

    def predict_plant(self, img_path):
        img = Image.open(img_path).convert("RGB")
        x = plant_transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            p = F.softmax(plant_model(x), 1)[0]
        i = torch.argmax(p)
        return plant_classes[i], float(p[i])

    def predict_insect(self, img_path):
        img = Image.open(img_path).convert("RGB")
        x = plant_transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            p = F.softmax(insect_model(x), 1)[0]
        i = torch.argmax(p)
        return INSECT_CLASSES[i], float(p[i])

system = AgroAI()


In [62]:
def auto_route(text=None, image_path=None, city=None):
    """
    Universal AgroAI Router
    Supports:
    - Image → CNN → Disease / Insect
    - Text → Disease / Insect / Deficiency
    """

    weather = get_weather(city) if city else "Weather not requested"

    # ==================================================
    # 1️⃣ IMAGE MODE (CNN HAS HIGHEST PRIORITY)
    # ==================================================
    if image_path:
        plant, plant_conf = system.predict_plant(image_path)
        insect, insect_conf = system.predict_insect(image_path)

        if plant_conf >= 0.5:
            return {
                "mode": "plant",
                "confidence": plant_conf,
                "plant_info": system.plant_rag.get_disease_block(
                    plant, CLASS_TO_TITLE
                ),
                "insect_info": {},
                "weather": weather
            }
        else:
            return {
                "mode": "insect",
                "confidence": insect_conf,
                "plant_info": {},
                "insect_info": system.insect_rag.query(insect),
                "weather": weather
            }

    # ==================================================
    # 2️⃣ TEXT MODE (NO IMAGE)
    # ==================================================
    if text:
        q = text.lower()

        # -----------------------------
        # 🌱 CROP DETECTION
        # -----------------------------
        crop = None
        if "tomato" in q:
            crop = "tomato"
        elif "potato" in q:
            crop = "potato"
        elif "pepper" in q:
            crop = "pepper"

        # ==================================================
        # 🌿 DISEASE DETECTION (TOP PRIORITY FOR TEXT)
        # ==================================================
        best_match = None
        best_score = 0

        for disease, info in KNOWLEDGE["Diseases"].items():

            # Crop-aware filtering
            if crop and crop not in disease.lower():
                continue

            symptoms = info.get("Symptoms", "").lower()

            score = sum(1 for w in q.split() if w in symptoms)

            if score > best_score:
                best_score = score
                best_match = (disease, info)

        if best_match and best_score >= 2:
            disease, info = best_match
            return {
                "mode": "plant",
                "confidence": min(0.85, 0.4 + best_score * 0.1),
                "plant_info": {**info, "name": disease},
                "insect_info": {},
                "weather": weather
            }

        # ==================================================
        # 🐛 INSECT DETECTION (STRICT, ENTITY-BASED)
        # ==================================================
        insect_keywords = [
            "aphid", "aphids", "mite", "mites",
            "whitefly", "whiteflies",
            "thrips", "worm", "worms",
            "beetle", "larvae", "webs", "insect", "bugs"
        ]

        if any(k in q for k in insect_keywords):
            insect_hits = []

            for insect, info in INSECT_KNOWLEDGE.items():
                name_tokens = insect.lower().split()
                symptom_text = info.get("Symptoms", "").lower()

                score = 0
                for w in q.split():
                    if w in name_tokens or w in symptom_text:
                        score += 1

                if score >= 2:
                    insect_hits.append((score, insect))

            if insect_hits:
                insect_hits.sort(reverse=True)
                insect_name = insect_hits[0][1]

                return {
                    "mode": "insect",
                    "confidence": 0.75,
                    "plant_info": {},
                    "insect_info": INSECT_KNOWLEDGE[insect_name],
                    "weather": weather
                }

        # ==================================================
        # 🧪 DEFICIENCY DETECTION
        # ==================================================
        deficiency_keywords = [
            "yellow", "chlorosis", "purple",
            "purplish", "interveinal",
            "veins", "pale"
        ]

        if any(w in q for w in deficiency_keywords):
            return {
                "mode": "deficiency",
                "confidence": 0.7,
                "plant_info": {},
                "insect_info": {},
                "extra": KNOWLEDGE["Deficiencies"],
                "weather": weather
            }

        # ==================================================
        # 🤖 TEXT FALLBACK (LLM-LIKE RESPONSE)
        # ==================================================
        return {
            "mode": "text",
            "confidence": None,
            "plant_info": {},
            "insect_info": {},
            "extra": {
                "answer": (
                    "I need a bit more information. "
                    "Please mention the crop name and visible symptoms "
                    "(spots, holes, insects, mold, color changes), "
                    "or upload an image for accurate diagnosis."
                )
            },
            "weather": weather
        }

    # ==================================================
    # 3️⃣ EMPTY INPUT
    # ==================================================
    return {
        "mode": "error",
        "confidence": None,
        "plant_info": {},
        "insect_info": {},
        "weather": weather,
        "message": "Please provide either an image or a text description."
    }


In [63]:
def generate_nlg_response(result):
    lines = []

    # 🟡 TEXT-ONLY RESPONSE
    if result["mode"] == "text":
        lines.append("🧠 AI Analysis")
        lines.append(result["answer"])
        return "\n".join(lines)

    # 🐛 INSECT RESPONSE
    if result["mode"] == "insect":
        conf = round(result["confidence"] * 100, 1)
        info = result["insect_info"]

        lines.append("🧠 AI Diagnosis Result")
        lines.append(f"Issue Detected: Insect Infestation ({conf}% confidence)\n")

        if "Symptoms" in info:
            lines.append(f"Symptoms:\n{info['Symptoms']}")
        if "Cause" in info:
            lines.append(f"\nCause:\n{info['Cause']}")

        lines.append("\nRecommended Action:")
        if "Organic Control" in info:
            lines.append(f"- Organic: {info['Organic Control']}")
        if "Prevention" in info:
            lines.append(f"- Prevention: {info['Prevention']}")

        return "\n".join(lines)

    # 🌱 PLANT DISEASE RESPONSE
    if result["mode"] == "plant":
        conf = round(result["confidence"] * 100, 1)
        info = result["plant_info"]

        lines.append("🧠 AI Diagnosis Result")
        lines.append(
            f"Disease Identified: {info.get('name','Unknown')} "
            f"({conf}% confidence)\n"
        )

        if "Symptoms" in info:
            lines.append(f"Symptoms:\n{info['Symptoms']}")
        if "Cause" in info:
            lines.append(f"\nCause:\n{info['Cause']}")

        lines.append("\nRecommended Treatment:")
        if "Organic Control" in info:
            lines.append(f"- Organic: {info['Organic Control']}")
        if "Chemical Control" in info:
            lines.append(f"- Chemical: {info['Chemical Control']}")
        if "Prevention" in info:
            lines.append(f"- Prevention: {info['Prevention']}")

        return "\n".join(lines)

    return "Unable to generate response."


In [64]:
img = r"K:\cnndisease\WhatsApp Image 2025-12-12 at 9.00.16 PM.jpeg"

result = auto_route(image_path=img)
text_output = generate_nlg_response(result)

print(text_output)


🧠 AI Diagnosis Result
Issue Detected: Insect Infestation (98.9% confidence)

Symptoms:
Holes in stems, frass deposits, poor grain fill.

Cause:
Larvae boring into corn stalks.

Recommended Action:
- Organic: Bt maize, Bt spray, parasitic wasps.
- Prevention: Destroy crop residue; pheromone traps.


In [53]:
def evaluate_rag_retrieval_accuracy(class_to_title, rag):
    correct = 0
    total = 0

    for cnn_class, expected_title in class_to_title.items():
        block = rag.get_disease_block(cnn_class, class_to_title)

        if block and block.get("name") == expected_title:
            correct += 1

        total += 1

    return (correct / total) * 100


In [54]:
rag_accuracy = evaluate_rag_retrieval_accuracy(CLASS_TO_TITLE, system.plant_rag)
print(f"✅ RAG Retrieval Accuracy: {rag_accuracy:.2f}%")


✅ RAG Retrieval Accuracy: 100.00%


In [45]:
def rag_faithfulness(response_block, rag_block):
    resp_words = set(json.dumps(response_block).lower().split())
    rag_words = set(json.dumps(rag_block).lower().split())

    if not resp_words:
        return 0.0

    return len(resp_words & rag_words) / len(resp_words)


In [ ]:
"K:\cnndisease\00b1f292-23dd-44d4-aad3-c1ffb6a6ad5a___RS_LB 4479.JPG"

In [55]:
img = r"K:\cnndisease\00b1f292-23dd-44d4-aad3-c1ffb6a6ad5a___RS_LB 4479.JPG"

result = auto_route(image_path=img)
text_output = generate_nlg_response(result)

print(text_output)


🧠 AI Diagnosis Result
Disease Identified: Potato Late Blight (91.2% confidence)

Symptoms:
Dark greasy lesions, white mold under leaves, tuber rot.

Cause:
Phytophthora infestans.

Recommended Treatment:
- Organic: Neem oil (4 ml/L), copper oxychloride (3 g/L).
- Chemical: Mancozeb, Metalaxyl.
- Prevention: Remove infected plants, avoid water stagnation.


In [65]:
result = auto_route(
    text="My tomato leaves have brown rings and yellowing"
)
print(generate_nlg_response(result))


🧠 AI Diagnosis Result
Disease Identified: Tomato Early Blight (70.0% confidence)

Symptoms:
Concentric brown rings, leaf yellowing.

Cause:
Alternaria solani.

Recommended Treatment:
- Organic: Compost tea, neem oil.
- Chemical: Mancozeb, chlorothalonil.
- Prevention: Remove infected leaves.


In [66]:
result = auto_route(
    text="My tomato leaves are curling and feel sticky. I also see ants."
)
print(generate_nlg_response(result))

🧠 AI Diagnosis Result
Disease Identified: Tomato Late Blight (60.0% confidence)

Symptoms:
Greasy lesions, white mold under leaves.

Cause:
Phytophthora infestans.

Recommended Treatment:
- Organic: Neem, copper oxychloride.
- Chemical: Mancozeb, Metalaxyl.
- Prevention: Remove infected plants.
